### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-64    3.3   21     0.37   1        Bom      0.95      
-56    1.9   40     0.07   1        Bom      1.00      
-78    6.2   99     0.57   0        Ruim     1.00      
-64    3.5   2      0.36   1        Bom      0.99      
-66    4.0   60     0.29   0        Ruim     0.87      
-59    4.0   17     0.35   1        Bom      0.96      
-50    3.2   36     0.05   1        Bom      1.00      
-79    3.9   94     0.45   0        Ruim     1.00      
-79    6.6   32     0.43   0        Ruim     0.99      
-49    3.6   21     0.33   1        Bom      0.98      
-59    2.8   35     0.25   1        Bom      0.98      
-84    9.4   72     0.64   0        Ruim     1.00      
-74    4.3   58     0.87   0        Ruim     1.00      
-61    6.8   39     0.82   0        Ruim     1.00      
-71    8.2   65     0.85   0        Ruim     1.00      
-60    1.7   4      0.17   1        Bom      1.0

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
